In [87]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

def generate_pendulum_animation(num_pendulums, lengths, initial_angles_deg, astro_obj, filename):
    # Imagine you have 3 swings (pendulums) in a playground. We're going to describe each one:
    N = num_pendulums  # Let's say we have 3 pendulums, like 3 swings.
    lengths = lengths  # e.g.: Each swing's chain is 0.1 meters long.
    initial_angles_deg = initial_angles_deg  # Each swing starts at a different angle: 20, 25, and 30 degrees away from straight down.
    
    initial_angles = np.radians(initial_angles_deg)
    
    # Approximate surface gravity values for all the planets in our solar system and our Moon.
    gravities_map = {
        'mercury' : 3.7,
        'venus' : 8.87,
        'earth' : 9.81,
        'mars' : 3.71,
        'jupiter' : 24.79,
        'saturn' : 10.44,
        'uranus' : 8.69,
        'neptune' : 11.15,
        'earth_moon' : 1.62,
        'sun' : 274,
    }

    # Define colors for each pendulum
    colors = ['blue', 'green', 'red', 'cyan', 'magenta'][:num_pendulums]
    
    # Swings move back and forth because of gravity, so we need to tell our simulation about it:
    g = gravities_map[astro_obj]  # Gravity makes things fall down. It's a number that helps us calculate how the swings move.
    
    # To make our swing simulation look right, we set how smooth and fast it looks:
    config_interval = 10  # This makes our simulation smooth and not too choppy.
    config_fps = 30  # This makes the swings move in a realistic way, not too fast or slow.
    
    # We want to see how the swings move over 20 seconds, so we set up a timeline:
    t = np.linspace(0, 20, 1000)  # This is like a stopwatch that takes a snapshot 1000 times over 20 seconds.
    
    # We want all swings to fit in our drawing, so we make sure there's enough space:
    max_length = max(lengths)  # Find the longest swing chain.
    plot_limit = max_length * (1 + 0.3)  # Make sure there's extra space so swings don't hit the edges of our drawing.
    
    # Now we start drawing:
    fig, ax = plt.subplots()  # This is like getting a piece of paper ready.
    ax.set_xlim(-plot_limit, plot_limit)  # This makes sure we draw the horizontal space big enough.
    ax.set_ylim(-plot_limit, plot_limit / 6)  # This makes sure we draw the vertical space big enough.
    ax.set_aspect('equal')  # This makes sure our drawing isn't stretched or squished.
    lines = [ax.plot([], [], 'o-', lw=2, color=colors[i])[0] for i in range(num_pendulums)]
    
    # Before we start the simulation, we make sure each swing is at its starting position:
    def init():
        for line in lines:
            line.set_data([], [])
        return lines
    
    # Now, for every snapshot in our 20-second timeline, we calculate where each swing should be:
    def animate(i):
        for j, line in enumerate(lines):
            L = lengths[j]  # We look at each swing's chain length.
            # When we calculate where the swings should be at each point in time, we're using a special formula:
            theta = initial_angles[j] * np.cos(np.sqrt(g / L) * t[i])
            # This line is like using a magic recipe to find out how far back or forward each swing has moved.
            # Here's what each part does:
            # - initial_angles[j]: This is where the swing started. Just like if you pulled a swing back and let go, where you pulled it to is important.
            # - np.cos(...): This part is about the swing's back-and-forth motion. Swings (and pendulums) move in a special way that repeats, which is why we use "cos," a math function that also repeats.
            # - np.sqrt(g / L) * t[i]: This bit tells us how fast the swing moves back and forth. It's based on two things:
            #     - g: Gravity, because it's what pulls the swing back down each time you push it.
            #     - L: How long the swing's chain is. Longer chains make the swing take longer to come back.
            #     - t[i]: This is our stopwatch. It tells us "when" we're looking at the swing. As time goes on, the swing moves to different positions.
            
            # The whole formula gives us an angle, telling us how much the swing has swung from the center (straight down) at each point in time.
            x, y = L * np.sin(theta), -L * np.cos(theta)
            # After we get the angle from our formula, we need to turn it into a spot on our drawing (our "paper"). 
            # We use "sin" and "cos" again for this because they help us take the swing's angle and figure out where it is horizontally (x) and vertically (y).
            # - L * np.sin(theta): This calculates how far to the left or right the bottom of the swing is.
            # - -L * np.cos(theta): This calculates how high or low the bottom of the swing is.
            # Together, x and y tell us exactly where to draw the swing on our paper at each moment.
            line.set_data([0, x], [0, y])  # We draw the swing in its new position.
        return lines

    periods = [2 * np.pi * np.sqrt(L / g) for L in lengths]
    frequencies = [1 / T for T in periods]
 
    # Now we put it all together and make the animation:
    ani = animation.FuncAnimation(fig, animate, frames=len(t), init_func=init, blit=True, interval=config_interval)

    # Add text for each pendulum's period and frequency
    for i, (p, f) in enumerate(zip(periods, frequencies)):
        plt.text(-plot_limit + 0.1, plot_limit/6 - 0.2*(i+1), f'Pendulum {i+1}: {p:.2f}s, {f:.2f}Hz', color=colors[i])
    
    # We save the animation so we can watch it:
    ani.save(filename, writer='pillow', fps=config_fps)
    
    # If we wanted to watch it right now, we'd use this:
    plt.close()

astro_obj = "earth"

generate_pendulum_animation(3, [1,1,1], [22, 45, 90], astro_obj, "pendulum_on_" +astro_obj+ ".gif")
